In [2]:
import os
import time
from datetime import datetime, timedelta

def get_files_with_timestamps(path):
    files_with_timestamps = {}
    for dirpath, dirnames, filenames in os.walk(path):
        for file in filenames:
            full_path = os.path.join(dirpath, file)
            files_with_timestamps[full_path] = os.path.getmtime(full_path)
    return files_with_timestamps


def monitor_directories(dirs):
    # Store the initial state of the directories
    prev_state = {dir: get_files_with_timestamps(dir) for dir in dirs}

    while True:
        # Sleep first to prevent excessive CPU usage
        time.sleep(5)

        print(f"Checking {len(prev_state)} folders with {sum([len(prev_state[dir]) for dir in prev_state])} files...")

        # Get the current state of the directories
        curr_state = {dir: get_files_with_timestamps(dir) for dir in dirs}

        for dir in dirs:
            # Determine added, removed and modified files
            added = [f for f in curr_state[dir] if f not in prev_state[dir]]
            removed = [f for f in prev_state[dir] if f not in curr_state[dir]]
            modified = [f for f in curr_state[dir] if f in prev_state[dir] and curr_state[dir][f] != prev_state[dir][f]]

            # If there are any changes, print them
            if added:
                print(f"In {dir} - Added: ", ", ".join(added))
            if removed:
                print(f"In {dir} - Removed: ", ", ".join(removed))
            if modified:
                print(f"In {dir} - Modified: ", ", ".join(modified))

            # Create a list of paths to add to the database
            to_add = added + modified

            if to_add:
                print(f"Adding {len(to_add)} files to the database...")

        # Update the stored state
        prev_state = curr_state

if __name__ == "__main__":
    # Calculate the last 60 days
    base_path = "/mnt/nas05/data01/radio/2002-20yy_Callisto/"
    days_to_check = 30
    dirs_to_monitor = []
    for i in range(days_to_check):
        day_ago = datetime.now() - timedelta(days=i)
        date_path = day_ago.strftime('%Y/%m/%d')
        dirs_to_monitor.append(f"{base_path}{date_path}/")
    monitor_directories(dirs_to_monitor)


Checking 30 folders with 120161 files...
Checking 30 folders with 120161 files...
Checking 30 folders with 120161 files...
Checking 30 folders with 120161 files...
Checking 30 folders with 120161 files...
